In [1]:
from lightgbm import LGBMClassifier, LGBMRegressor
import optuna
import warnings
warnings.filterwarnings("ignore")
from get_vec_function_translate import get_vec6
import copy
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score
from chembench import dataset
import torch
print(torch.cuda.is_available())
import deepchem as dc
from chembench import load_data
dc.__version__

RDKit WARNING: [09:55:51] Enabling RDKit 2019.09.3 jupyter extensions
[09:55:51] Enabling RDKit 2019.09.3 jupyter extensions


True


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


'2.6.1'

In [2]:
epoch = 'new_trfm_98_10_translate'

model_weight = 'trfm_new_98_10000.pkl'

In [4]:
def objective_regressor(trial):
    model_name = trial.suggest_categorical('regressor', [ 'LGBM']) 
    if model_name == 'LGBM':
        num_leaves = trial.suggest_int('num_leaves', 5, 100)
        min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 100)
        max_depth = trial.suggest_int('max_depth', -1, 10)
        lr = trial.suggest_loguniform('lr', 1e-5, 1.0)
        max_bin = trial.suggest_int('max_bin', 10, 500)
        reg = LGBMRegressor(num_leaves=num_leaves, min_data_in_leaf=min_data_in_leaf, max_depth=max_depth,
                           learning_rate=lr, max_bin=max_bin)
    
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_val)
    return mean_squared_error(y_pred, y_val)**0.5

def evaluate_regression(model, X_train, X_val, X_test, y_train, y_val, y_test):
    reg = copy.deepcopy(model)
    reg.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
    y_pred = reg.predict(X_test)
    rmse = mean_squared_error(y_pred, y_test)**0.5
    r2 = r2_score(y_pred, y_test)
    return rmse, r2

In [5]:
def vec_ave(data,hidden_size):
    data.columns = range(-1,2*hidden_size,1)
    data1 = data.iloc[:,:hidden_size+1]
    data2 = data.iloc[:,hidden_size+1:]
    data1.columns = range(-1,hidden_size,1)
    data1 = data1.rename(columns={-1:'ids'})
    data2.columns= range(0,hidden_size,1)
    data2.insert(loc=0, column='ids', value=data[-1])
    data3 = (data1.iloc[:,1:hidden_size+1])/2 + (data2.iloc[:,1:hidden_size+1])/ 2
    # data4 = normalize(data3, axis=1)
    DATA = pd.DataFrame(data3)
    # DATA.insert(loc=0, column='ids', value=data[-1])
    return DATA

In [51]:
def objective_classifier(trial):
    model_name = trial.suggest_categorical('classifier', [ 'LGBM']) 
    if model_name == 'LGBM':
        num_leaves = trial.suggest_int('num_leaves', 5, 100)
        min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 100)
        max_depth = trial.suggest_int('max_depth', -1, 10)
        lr = trial.suggest_loguniform('lr', 1e-5, 1.0)
        max_bin = trial.suggest_int('max_bin', 10, 500)
        clf = LGBMClassifier(num_leaves=num_leaves, min_data_in_leaf=min_data_in_leaf, max_depth=max_depth,
                           learning_rate=lr, max_bin=max_bin)
    
    clf.fit(X_train, y_train)
    y_score = clf.predict_proba(X_val)
    return - roc_auc_score(y_val, y_score[:,1])

def evaluate_classification(model, X_train, X_val, X_test, y_train, y_val, y_test):
    clf = copy.deepcopy(model)
    clf.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
    y_score = clf.predict_proba(X_test)
    auc = roc_auc_score(y_test, y_score[:,1])
    return auc

In [ ]:
# requires X, y
def objective_classifier_multi(trial):
    model_name = trial.suggest_categorical('classifier', [ 'LGBM'])
    if model_name == 'LGBM':
        num_leaves = trial.suggest_int('num_leaves', 5, 100)
        min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 100)
        max_depth = trial.suggest_int('max_depth', -1, 10)
        lr = trial.suggest_loguniform('lr', 1e-5, 1.0)
        max_bin = trial.suggest_int('max_bin', 10, 500)
        clf = LGBMClassifier(num_leaves=num_leaves, min_data_in_leaf=min_data_in_leaf, max_depth=max_depth,
                           learning_rate=lr, max_bin=max_bin, n_jobs=16)
    
    aucs = np.empty(len(KEYS))
    for i,key in enumerate(KEYS):
        _clf = copy.deepcopy(clf)
        _clf.fit(X_train[y_train[key].notna()], y_train[key].dropna().values)
        y_score = _clf.predict_proba(X_val[y_val[key].notna()])
        aucs[i] = roc_auc_score(y_val[key].dropna().values, y_score[:,1])
    return - np.mean(aucs)


from tqdm import tqdm_notebook as tqdm
def evaluate_classification_multi(model, X_train, X_val, X_test):
    aucs = np.empty(len(KEYS))
    for i,key in tqdm(enumerate(KEYS)):
        clf = copy.deepcopy(model)
        _X_train = np.concatenate([X_train[y_train[key].notna()], X_val[y_val[key].notna()]])
        _y_train = np.concatenate([y_train[key].dropna().values, y_val[key].dropna().values])
        clf.fit(_X_train, _y_train)
        y_score = clf.predict_proba(X_test[y_test[key].notna()])
        aucs[i] = roc_auc_score(y_test[key].dropna().values, y_score[:,1])
    return np.mean(aucs)

def vec_ave(data):
    data.columns = range(-1,512,1)
    data1 = data.iloc[:,:257]
    data2 = data.iloc[:,257:]
    data1.columns = range(-1,256,1)
    data1 = data1.rename(columns={-1:'ids'})
    data2.columns= range(0,256,1)
    data2.insert(loc=0, column='ids', value=data[-1])
    data3 = (data1.iloc[:,1:257])/2 + (data2.iloc[:,1:257])/ 2
    # data4 = normalize(data3, axis=1)
    DATA = pd.DataFrame(data3)
    # DATA.insert(loc=0, column='ids', value=data[-1])
    return DATA

param = []
aucs = [[],[],[]]

optuna

In [6]:
esol = dataset.load_ESOL()
frs = dataset.load_FreeSolv()
lipo = dataset.load_Lipop()
pdbf = dataset.load_PDBF()
pdbc = dataset.load_PDBC()
pdbr = dataset.load_PDBR()
datasets = [esol,frs,lipo,pdbf,pdbc,pdbr]

param = []
rmses,r2s = [[],[],[],[],[],[]],[[],[],[],[],[],[]]

total samples: 1128
total samples: 642
total samples: 4200
total samples: 9880
total samples: 168
total samples: 3040


ESOL

In [15]:
i = 0
data = datasets[i]
rmses[i] = []
r2s[i] = []
task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)

X = get_vec6(smiles,data=task_name,model_weight=model_weight,hidden_size=256,epoch=epoch,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_regressor, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)


    if study.best_params['regressor']=='LGBM':
            rmse, r2 = evaluate_regression(LGBMRegressor(num_leaves=study.best_params['num_leaves'], min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                            max_depth=study.best_params['max_depth'],
                                            learning_rate=study.best_params['lr'], 
                                            max_bin=study.best_params['max_bin']),
                                            X_train, X_val, X_test, y_train, y_val, y_test)

    
    rmses[i].append(rmse)
    r2s[i].append(r2)


loading dataset: ESOL number of split times: 3


100%|██████████| 1128/1128 [00:00<00:00, 52806.24it/s]

Built smi corpus file!



100%|██████████| 141/141 [00:01<00:00, 126.55it/s]
[I 2023-07-10 10:14:13,170] A new study created in memory with name: no-name-fef9714a-9145-451f-af7e-9152e432df1d


[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=29


...

[LightGBM] [Warning] min_data_in_leaf is set=57, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=57
[LightGBM] [Warning] min_data_in_leaf is set=60, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=60


[I 2023-07-10 10:15:23,425] Trial 97 finished with value: 0.8305264364139514 and parameters: {'regressor': 'LGBM', 'num_leaves': 91, 'min_data_in_leaf': 60, 'max_depth': 5, 'lr': 0.027295477065523002, 'max_bin': 173}. Best is trial 72 with value: 0.6988082105367144.
[I 2023-07-10 10:15:23,589] Trial 98 finished with value: 0.740435198861196 and parameters: {'regressor': 'LGBM', 'num_leaves': 98, 'min_data_in_leaf': 70, 'max_depth': 2, 'lr': 0.1493722122700598, 'max_bin': 500}. Best is trial 72 with value: 0.6988082105367144.


[LightGBM] [Warning] min_data_in_leaf is set=70, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=70
[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=67


[I 2023-07-10 10:15:23,875] Trial 99 finished with value: 0.7069427046535421 and parameters: {'regressor': 'LGBM', 'num_leaves': 58, 'min_data_in_leaf': 67, 'max_depth': 9, 'lr': 0.07691387766198077, 'max_bin': 461}. Best is trial 72 with value: 0.6988082105367144.


{'regressor': 'LGBM', 'num_leaves': 92, 'min_data_in_leaf': 59, 'max_depth': 3, 'lr': 0.1251065808188638, 'max_bin': 484}
[LightGBM] [Warning] min_data_in_leaf is set=59, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=59


In [16]:
print(rmses[i])
print(param[-3:])
print('RMSE_ESOL | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(rmses[i]), np.std(rmses[i])))
print('R2_ESOL   | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(r2s[i]), np.std(r2s[i])))

[0.7527167454442767, 0.675425802187474, 0.6098541876058983]
[{'regressor': 'LGBM', 'num_leaves': 29, 'min_data_in_leaf': 13, 'max_depth': 5, 'lr': 0.07542594048844199, 'max_bin': 263}, {'regressor': 'LGBM', 'num_leaves': 93, 'min_data_in_leaf': 49, 'max_depth': 10, 'lr': 0.2061078837211253, 'max_bin': 478}, {'regressor': 'LGBM', 'num_leaves': 92, 'min_data_in_leaf': 59, 'max_depth': 3, 'lr': 0.1251065808188638, 'max_bin': 484}]
RMSE_ESOL | MEAN: 0.6793 | STD: 0.0584
R2_ESOL   | MEAN: 0.8630 | STD: 0.0357


Lipo

In [17]:
i = 2
data = datasets[i]
rmses[i] = []
r2s[i] = []
task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)

X = get_vec6(smiles,data=task_name,model_weight=model_weight,hidden_size=256,epoch=epoch,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_regressor, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)


    if study.best_params['regressor']=='LGBM':
            rmse, r2 = evaluate_regression(LGBMRegressor(num_leaves=study.best_params['num_leaves'], min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                            max_depth=study.best_params['max_depth'],
                                            learning_rate=study.best_params['lr'], 
                                            max_bin=study.best_params['max_bin']),
                                            X_train, X_val, X_test, y_train, y_val, y_test)

    rmses[i].append(rmse)
    r2s[i].append(r2)

loading dataset: Lipop number of split times: 3


100%|██████████| 4200/4200 [00:00<00:00, 28026.62it/s]

Built smi corpus file!



100%|██████████| 525/525 [00:04<00:00, 108.47it/s]
[I 2023-07-10 10:17:09,479] A new study created in memory with name: no-name-13f6fc82-25f3-49d9-9503-f6de7da802f2


[LightGBM] [Warning] min_data_in_leaf is set=24, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=24


...

[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86


[I 2023-07-10 10:20:32,985] Trial 98 finished with value: 0.8248235160209719 and parameters: {'regressor': 'LGBM', 'num_leaves': 53, 'min_data_in_leaf': 86, 'max_depth': 10, 'lr': 0.06886182790871097, 'max_bin': 35}. Best is trial 82 with value: 0.7654643306086603.


[LightGBM] [Warning] min_data_in_leaf is set=60, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=60


[I 2023-07-10 10:20:33,486] Trial 99 finished with value: 1.0903530386535967 and parameters: {'regressor': 'LGBM', 'num_leaves': 71, 'min_data_in_leaf': 60, 'max_depth': 9, 'lr': 0.0043795585386909395, 'max_bin': 86}. Best is trial 82 with value: 0.7654643306086603.


{'regressor': 'LGBM', 'num_leaves': 60, 'min_data_in_leaf': 74, 'max_depth': 10, 'lr': 0.16898383398440314, 'max_bin': 54}
[LightGBM] [Warning] min_data_in_leaf is set=74, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=74


In [18]:
print(rmses[i])
print(param[-3:])
print('RMSE_lipo | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(rmses[i]), np.std(rmses[i])))
print('R2_lipo   | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(r2s[i]), np.std(r2s[i])))

[0.7612859272656851, 0.7690710993164515, 0.80120036783972]
[{'regressor': 'LGBM', 'num_leaves': 25, 'min_data_in_leaf': 45, 'max_depth': -1, 'lr': 0.08874116439069638, 'max_bin': 343}, {'regressor': 'LGBM', 'num_leaves': 82, 'min_data_in_leaf': 79, 'max_depth': 10, 'lr': 0.12654895247816847, 'max_bin': 117}, {'regressor': 'LGBM', 'num_leaves': 60, 'min_data_in_leaf': 74, 'max_depth': 10, 'lr': 0.16898383398440314, 'max_bin': 54}]
RMSE_lipo | MEAN: 0.7772 | STD: 0.0173
R2_lipo   | MEAN: 0.1557 | STD: 0.0439


frsl

In [29]:
i = 1
data = datasets[i]
rmses[i] = []
r2s[i] = []
task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)

X = get_vec6(smiles,data=task_name,model_weight=model_weight,hidden_size=256,epoch=epoch,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_regressor, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)


    if study.best_params['regressor']=='LGBM':
            rmse, r2 = evaluate_regression(LGBMRegressor(num_leaves=study.best_params['num_leaves'], min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                            max_depth=study.best_params['max_depth'],
                                            learning_rate=study.best_params['lr'], 
                                            max_bin=study.best_params['max_bin']),
                                            X_train, X_val, X_test, y_train, y_val, y_test)
    
    rmses[i].append(rmse)
    r2s[i].append(r2)

loading dataset: FreeSolv number of split times: 3


100%|██████████| 642/642 [00:00<00:00, 66660.31it/s]


Built smi corpus file!


100%|██████████| 81/81 [00:00<00:00, 125.64it/s]
[I 2023-07-10 10:33:24,697] A new study created in memory with name: no-name-00efc871-de19-4f6d-b800-35bccbd46abc


[LightGBM] [Warning] min_data_in_leaf is set=27, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=27


...

[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
[LightGBM] [Warning] min_data_in_leaf is set=70, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=70


[I 2023-07-10 10:34:07,870] Trial 97 finished with value: 1.8652620760313947 and parameters: {'regressor': 'LGBM', 'num_leaves': 44, 'min_data_in_leaf': 69, 'max_depth': 6, 'lr': 0.38071411551266743, 'max_bin': 452}. Best is trial 78 with value: 1.7447848991107224.


[LightGBM] [Warning] min_data_in_leaf is set=69, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=69
[LightGBM] [Warning] min_data_in_leaf is set=47, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=47


[I 2023-07-10 10:34:07,974] Trial 98 finished with value: 2.059108538765416 and parameters: {'regressor': 'LGBM', 'num_leaves': 54, 'min_data_in_leaf': 47, 'max_depth': 7, 'lr': 0.0705094575744144, 'max_bin': 22}. Best is trial 78 with value: 1.7447848991107224.
[I 2023-07-10 10:34:08,125] Trial 99 finished with value: 2.0659330399684532 and parameters: {'regressor': 'LGBM', 'num_leaves': 57, 'min_data_in_leaf': 56, 'max_depth': 6, 'lr': 0.7805427108075658, 'max_bin': 494}. Best is trial 78 with value: 1.7447848991107224.


[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56
{'regressor': 'LGBM', 'num_leaves': 32, 'min_data_in_leaf': 48, 'max_depth': 10, 'lr': 0.24093094864736123, 'max_bin': 17}
[LightGBM] [Warning] min_data_in_leaf is set=48, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=48


In [30]:
print(rmses[i])
print(param[-3:])
print('RMSE_frsl | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(rmses[i]), np.std(rmses[i])))
print('R2_frsl   | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(r2s[i]), np.std(r2s[i])))

[1.4288577335038484, 1.3022232773266391, 1.4136761877010933]
[{'regressor': 'LGBM', 'num_leaves': 19, 'min_data_in_leaf': 11, 'max_depth': 7, 'lr': 0.0726165621611161, 'max_bin': 476}, {'regressor': 'LGBM', 'num_leaves': 24, 'min_data_in_leaf': 13, 'max_depth': 7, 'lr': 0.2161549573551784, 'max_bin': 97}, {'regressor': 'LGBM', 'num_leaves': 32, 'min_data_in_leaf': 48, 'max_depth': 10, 'lr': 0.24093094864736123, 'max_bin': 17}]
RMSE_frsl | MEAN: 1.3816 | STD: 0.0565
R2_frsl   | MEAN: 0.8407 | STD: 0.0083


pdbc

In [41]:
i = 4
data = datasets[i]
rmses[i] = []
r2s[i] = []
task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)

X = get_vec6(smiles,data=task_name,model_weight=model_weight,hidden_size=256,epoch=epoch,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_regressor, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)


    if study.best_params['regressor']=='LGBM':
            rmse, r2 = evaluate_regression(LGBMRegressor(num_leaves=study.best_params['num_leaves'], min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                            max_depth=study.best_params['max_depth'],
                                            learning_rate=study.best_params['lr'], 
                                            max_bin=study.best_params['max_bin']),
                                            X_train, X_val, X_test, y_train, y_val, y_test)

    
    rmses[i].append(rmse)
    r2s[i].append(r2)

loading dataset: PDBbind-core number of split times: 1


100%|██████████| 168/168 [00:00<00:00, 19934.45it/s]

Built smi corpus file!


100%|██████████| 21/21 [00:00<00:00, 119.96it/s]
[I 2023-07-10 10:01:26,814] A new study created in memory with name: no-name-f6ffb46b-d5fd-45e2-875c-ca8591cbcb71
[I 2023-07-10 10:01:26,855] Trial 0 finished with value: 0.8098754496537028 and parameters: {'regressor': 'LGBM', 'num_leaves': 31, 'min_data_in_leaf': 51, 'max_depth': 0, 'lr': 0.049451058493972876, 'max_bin': 411}. Best is trial 0 with value: 0.8098754496537028.
[I 2023-07-10 10:01:26,910] Trial 1 finished with value: 0.7649984517917925 and parameters: {'regressor': 'LGBM', 'num_leaves': 91, 'min_data_in_leaf': 19, 'max_depth': 4, 'lr': 0.6615631103956549, 'max_bin': 312}. Best is trial 1 with value: 0.7649984517917925.
[I 2023-07-10 10:01:26,927] Trial 2 finished with value: 1.0125749549100393 and parameters: {'regressor': 'LGBM', 'num_leaves': 67, 'min_data_in_leaf': 87, 'max_depth': 6, 'lr': 0.017169089906445723, 'max_bin': 157}. Best is trial 1 with value: 0.7649984517917925.
[I 2023-07-10 10:01:26,990] Trial 3 finished

[LightGBM] [Warning] min_data_in_leaf is set=51, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=51
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] min_data_in_leaf is set=87, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=87
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] min_data_in_leaf is set=49, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=49


...

[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] min_data_in_leaf is set=38, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=38
[LightGBM] [Warning] min_data_in_leaf is set=35, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=35
[LightGBM] [Warning] min_data_in_leaf is set=43, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=43


[I 2023-07-10 10:01:31,945] Trial 94 finished with value: 0.7967215588925266 and parameters: {'regressor': 'LGBM', 'num_leaves': 13, 'min_data_in_leaf': 43, 'max_depth': 8, 'lr': 0.16708727124301828, 'max_bin': 238}. Best is trial 1 with value: 0.7649984517917925.
[I 2023-07-10 10:01:32,003] Trial 95 finished with value: 0.7992901618654604 and parameters: {'regressor': 'LGBM', 'num_leaves': 31, 'min_data_in_leaf': 47, 'max_depth': 5, 'lr': 0.04824511603012933, 'max_bin': 77}. Best is trial 1 with value: 0.7649984517917925.
[I 2023-07-10 10:01:32,067] Trial 96 finished with value: 0.8652964430209419 and parameters: {'regressor': 'LGBM', 'num_leaves': 28, 'min_data_in_leaf': 30, 'max_depth': 7, 'lr': 0.11217835236668904, 'max_bin': 173}. Best is trial 1 with value: 0.7649984517917925.
[I 2023-07-10 10:01:32,125] Trial 97 finished with value: 0.8076005280608937 and parameters: {'regressor': 'LGBM', 'num_leaves': 37, 'min_data_in_leaf': 44, 'max_depth': 8, 'lr': 0.08135684610732902, 'max_b

[LightGBM] [Warning] min_data_in_leaf is set=47, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=47
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=44, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=44
[LightGBM] [Warning] min_data_in_leaf is set=51, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=51


[I 2023-07-10 10:01:32,181] Trial 98 finished with value: 1.111248504766759 and parameters: {'regressor': 'LGBM', 'num_leaves': 40, 'min_data_in_leaf': 51, 'max_depth': 7, 'lr': 0.95817710149798, 'max_bin': 352}. Best is trial 1 with value: 0.7649984517917925.
[I 2023-07-10 10:01:32,239] Trial 99 finished with value: 0.8517333785803711 and parameters: {'regressor': 'LGBM', 'num_leaves': 48, 'min_data_in_leaf': 58, 'max_depth': 10, 'lr': 0.21737004049751663, 'max_bin': 304}. Best is trial 1 with value: 0.7649984517917925.


[LightGBM] [Warning] min_data_in_leaf is set=58, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=58
{'regressor': 'LGBM', 'num_leaves': 91, 'min_data_in_leaf': 19, 'max_depth': 4, 'lr': 0.6615631103956549, 'max_bin': 312}
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19


In [42]:
i = 4
print(rmses[i])
print(param[-1])
print('RMSE_PDBC   | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(rmses[i]), np.std(rmses[i])))
print('R2_PDBC     | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(r2s[i]), np.std(r2s[i])))

[0.9017910838076787]
{'regressor': 'LGBM', 'num_leaves': 91, 'min_data_in_leaf': 19, 'max_depth': 4, 'lr': 0.6615631103956549, 'max_bin': 312}
RMSE_PDBC   | MEAN: 0.9018 | STD: 0.0000
R2_PDBC     | MEAN: -0.4298 | STD: 0.0000


pdbf

In [11]:
i = 3
data = datasets[i]
rmses[i] = []
r2s[i] = []
task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)

X = get_vec6(smiles,data=task_name,model_weight=model_weight,epoch=epoch,hidden_size=256,n_head=4,n_layer=2)
X_ave = X.copy()


for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_regressor, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)


    if study.best_params['regressor']=='LGBM':
            rmse, r2 = evaluate_regression(LGBMRegressor(num_leaves=study.best_params['num_leaves'], min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                            max_depth=study.best_params['max_depth'],
                                            learning_rate=study.best_params['lr'], 
                                            max_bin=study.best_params['max_bin']),
                                            X_train, X_val, X_test, y_train, y_val, y_test)

    
    rmses[i].append(rmse)
    r2s[i].append(r2)
        


loading dataset: PDBbind-full number of split times: 1


  0%|          | 0/9880 [00:00<?, ?it/s]

100%|██████████| 9880/9880 [00:00<00:00, 19737.75it/s]


Built smi corpus file!


100%|██████████| 1235/1235 [00:12<00:00, 97.33it/s]
[I 2023-06-28 14:14:50,589] A new study created in memory with name: no-name-2c1f6c27-b361-4f29-9ecb-54e5760dd449


...

[LightGBM] [Warning] min_data_in_leaf is set=91, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=91


[I 2023-06-28 14:16:31,643] Trial 98 finished with value: 0.7268416362210591 and parameters: {'regressor': 'LGBM', 'num_leaves': 14, 'min_data_in_leaf': 91, 'max_depth': 9, 'lr': 0.04496024269313398, 'max_bin': 403}. Best is trial 47 with value: 0.7232898285636647.


[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79


[I 2023-06-28 14:16:32,674] Trial 99 finished with value: 0.7626605512058131 and parameters: {'regressor': 'LGBM', 'num_leaves': 13, 'min_data_in_leaf': 79, 'max_depth': 10, 'lr': 0.015473643927245337, 'max_bin': 403}. Best is trial 47 with value: 0.7232898285636647.


{'regressor': 'LGBM', 'num_leaves': 41, 'min_data_in_leaf': 60, 'max_depth': 0, 'lr': 0.03679415579800442, 'max_bin': 60}
[LightGBM] [Warning] min_data_in_leaf is set=60, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=60


In [12]:
i=3
print(rmses[i])
print(param[-1])
print('RMSE_PBDF | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(rmses[i]), np.std(rmses[i])))
print('R2_PDBF   | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(r2s[i]), np.std(r2s[i])))

[0.7132816028198629]
{'regressor': 'LGBM', 'num_leaves': 41, 'min_data_in_leaf': 60, 'max_depth': 0, 'lr': 0.03679415579800442, 'max_bin': 60}
RMSE_PBDF | MEAN: 0.7133 | STD: 0.0000
R2_PDBF   | MEAN: -0.7899 | STD: 0.0000


In [23]:
i = 5
data = datasets[i]
while True:
    rmses[i] = []
    r2s[i] = []
    task_name = data.task_name
    task_type = data.task_type
    _, induces = load_data(task_name)
    smiles = data.x
    Y = pd.DataFrame(data.y)

    X = get_vec6(smiles,data=task_name,model_weight=model_weight,epoch=epoch,hidden_size=256,n_head=4,n_layer=2)
    X_ave = X.copy()

    for e,idx in enumerate(induces):
        train_idx, valid_idx, test_idx = idx
        X_train = X_ave.loc[train_idx]
        X_val = X_ave.loc[valid_idx]
        X_test = X_ave.loc[test_idx]

        y_train = Y.loc[train_idx]
        y_val = Y.loc[valid_idx]
        y_test = Y.loc[test_idx]

        study = optuna.create_study()
        study.optimize(objective_regressor, n_trials=100, n_jobs=1)
        print(study.best_params)
        param.append(study.best_params)


        if study.best_params['regressor']=='LGBM':
                rmse, r2 = evaluate_regression(LGBMRegressor(num_leaves=study.best_params['num_leaves'], min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                                max_depth=study.best_params['max_depth'],
                                                learning_rate=study.best_params['lr'], 
                                                max_bin=study.best_params['max_bin']),
                                                X_train, X_val, X_test, y_train, y_val, y_test)
        
        rmses[i].append(rmse)
        r2s[i].append(r2)
        
    if np.mean(rmses[i])<0.930:
        break

loading dataset: PDBbind-refined number of split times: 1


100%|██████████| 3040/3040 [00:00<00:00, 25366.02it/s]

Built smi corpus file!



100%|██████████| 380/380 [00:03<00:00, 118.20it/s]
[I 2023-07-10 19:54:54,320] A new study created in memory with name: no-name-d8fab71c-81d8-482f-b3e4-8ab826dd406e


[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88


...

[LightGBM] [Warning] min_data_in_leaf is set=89, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=89


[I 2023-07-10 19:55:36,787] Trial 98 finished with value: 0.7977969706111879 and parameters: {'regressor': 'LGBM', 'num_leaves': 51, 'min_data_in_leaf': 89, 'max_depth': 10, 'lr': 0.09001882985509131, 'max_bin': 460}. Best is trial 35 with value: 0.7771195416898545.


[LightGBM] [Warning] min_data_in_leaf is set=27, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=27


[I 2023-07-10 19:55:38,249] Trial 99 finished with value: 0.788016998389609 and parameters: {'regressor': 'LGBM', 'num_leaves': 47, 'min_data_in_leaf': 27, 'max_depth': 9, 'lr': 0.026852522501498943, 'max_bin': 403}. Best is trial 35 with value: 0.7771195416898545.


{'regressor': 'LGBM', 'num_leaves': 32, 'min_data_in_leaf': 95, 'max_depth': 9, 'lr': 0.10171972063230932, 'max_bin': 96}
[LightGBM] [Warning] min_data_in_leaf is set=95, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=95


In [24]:
i=5
print(rmses[i])
print(param[-1])
print('RMSE_PDBR | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(rmses[i]), np.std(rmses[i])))
print('R2_PDBR   | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(r2s[i]), np.std(r2s[i])))

[0.859172228326859]
{'regressor': 'LGBM', 'num_leaves': 32, 'min_data_in_leaf': 95, 'max_depth': 9, 'lr': 0.10171972063230932, 'max_bin': 96}
RMSE_PDBR | MEAN: 0.8592 | STD: 0.0000
R2_PDBR   | MEAN: -0.5811 | STD: 0.0000


classification

In [50]:
hiv = dataset.load_HIV()
bace = dataset.load_BACE()
bbbp = dataset.load_BBBP()

datasets = [hiv, bace, bbbp]

total samples: 41127
total samples: 1513
total samples: 2039


In [33]:
def objective_classifier(trial):
    model_name = trial.suggest_categorical('classifier', [ 'LGBM'])
    if model_name ==  'LGBM':
        num_leaves = trial.suggest_int('num_leaves', 5, 100)
        min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 100)
        max_depth = trial.suggest_int('max_depth', -1, 10)
        lr = trial.suggest_loguniform('lr', 1e-5, 1.0)
        max_bin = trial.suggest_int('max_bin', 10, 500)
        clf = LGBMClassifier(num_leaves=num_leaves, min_data_in_leaf=min_data_in_leaf, max_depth=max_depth,
                           learning_rate=lr, max_bin=max_bin)
    
    clf.fit(X_train, y_train)
    y_score = clf.predict_proba(X_val)
    return - roc_auc_score(y_val, y_score[:,1])

def evaluate_classification(model, X_train, X_val, X_test, y_train, y_val, y_test):
    clf = copy.deepcopy(model)
    clf.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
    y_score = clf.predict_proba(X_test)
    auc = roc_auc_score(y_test, y_score[:,1])
    return auc

In [34]:
def vec_ave(data):
    data.columns = range(-1,512,1)
    data1 = data.iloc[:,:257]
    data2 = data.iloc[:,257:]
    data1.columns = range(-1,256,1)
    data1 = data1.rename(columns={-1:'ids'})
    data2.columns= range(0,256,1)
    data2.insert(loc=0, column='ids', value=data[-1])
    data3 = (data1.iloc[:,1:257])/2 + (data2.iloc[:,1:257])/ 2
    # data4 = normalize(data3, axis=1)
    DATA = pd.DataFrame(data3)
    # DATA.insert(loc=0, column='ids', value=data[-1])
    return DATA

param = []
aucs = [[],[],[]]

In [53]:
i = 0
data = datasets[i]
aucs[i] = []

task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)

X = get_vec6(smiles,data=task_name,model_weight=model_weight,epoch=epoch,hidden_size=256,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_classifier, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)

    if study.best_params['classifier']=='LGBM':
        auc = evaluate_classification(LGBMClassifier(num_leaves=study.best_params['num_leaves'], 
                                        min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                        max_depth=study.best_params['max_depth'],
                                        learning_rate=study.best_params['lr'], 
                                        max_bin=study.best_params['max_bin']),
                                        X_train, X_val, X_test, y_train, y_val, y_test)


    aucs[i].append(auc)

loading dataset: HIV number of split times: 3


100%|██████████| 41127/41127 [00:01<00:00, 30226.52it/s]


Built smi corpus file!


100%|██████████| 5141/5141 [01:59<00:00, 42.95it/s]
[I 2023-07-10 16:28:37,682] A new study created in memory with name: no-name-24631410-cac1-43e8-9f7e-02d89ef2b6fe


[LightGBM] [Warning] min_data_in_leaf is set=35, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=35


...

[LightGBM] [Warning] min_data_in_leaf is set=59, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=59


[I 2023-07-10 16:42:10,336] Trial 98 finished with value: -0.8328862923770332 and parameters: {'classifier': 'LGBM', 'num_leaves': 59, 'min_data_in_leaf': 59, 'max_depth': 9, 'lr': 0.05610761507511095, 'max_bin': 266}. Best is trial 73 with value: -0.8513741916519693.


[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56


[I 2023-07-10 16:42:13,673] Trial 99 finished with value: -0.8081275720164609 and parameters: {'classifier': 'LGBM', 'num_leaves': 54, 'min_data_in_leaf': 56, 'max_depth': 10, 'lr': 0.074759979703056, 'max_bin': 253}. Best is trial 73 with value: -0.8513741916519693.


{'classifier': 'LGBM', 'num_leaves': 68, 'min_data_in_leaf': 48, 'max_depth': 10, 'lr': 0.054816258539195445, 'max_bin': 91}
[LightGBM] [Warning] min_data_in_leaf is set=48, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=48


In [54]:
print(aucs[i])
print(param[-3:])
print('AUC_HIV  | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(aucs[i]), np.std(aucs[i])))

[0.7887464029819039, 0.7826667181675969, 0.7755306205218332]
[{'classifier': 'LGBM', 'num_leaves': 37, 'min_data_in_leaf': 83, 'max_depth': 10, 'lr': 0.041992452289667644, 'max_bin': 156}, {'classifier': 'LGBM', 'num_leaves': 62, 'min_data_in_leaf': 55, 'max_depth': 5, 'lr': 0.07242040467428743, 'max_bin': 109}, {'classifier': 'LGBM', 'num_leaves': 68, 'min_data_in_leaf': 48, 'max_depth': 10, 'lr': 0.054816258539195445, 'max_bin': 91}]
AUC_HIV  | MEAN: 0.7823 | STD: 0.0054


BBBP

In [57]:
i = 2
data = datasets[i]
aucs[i] = []

task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)

X = get_vec6(smiles,data=task_name,model_weight=model_weight,hidden_size=256,epoch=epoch,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_classifier, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)

    if study.best_params['classifier']=='LGBM':
        auc = evaluate_classification(LGBMClassifier(num_leaves=study.best_params['num_leaves'], 
                                        min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                        max_depth=study.best_params['max_depth'],
                                        learning_rate=study.best_params['lr'], 
                                        max_bin=study.best_params['max_bin']),
                                        X_train, X_val, X_test, y_train, y_val, y_test)

    aucs[i].append(auc)

loading dataset: BBBP number of split times: 3


  0%|          | 0/2039 [00:00<?, ?it/s]

100%|██████████| 2039/2039 [00:00<00:00, 21574.15it/s]

Built smi corpus file!



100%|██████████| 255/255 [00:02<00:00, 123.42it/s]
[I 2023-07-10 16:45:14,118] A new study created in memory with name: no-name-618428a9-8c6d-4deb-9706-4ba283beebe5


...

[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32


[I 2023-07-10 16:47:42,449] Trial 98 finished with value: -0.9920419254658385 and parameters: {'classifier': 'LGBM', 'num_leaves': 82, 'min_data_in_leaf': 32, 'max_depth': 9, 'lr': 0.06351183637813354, 'max_bin': 111}. Best is trial 62 with value: -0.9952445652173914.


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


[I 2023-07-10 16:47:42,920] Trial 99 finished with value: -0.9950504658385093 and parameters: {'classifier': 'LGBM', 'num_leaves': 88, 'min_data_in_leaf': 30, 'max_depth': 8, 'lr': 0.08377489763718635, 'max_bin': 125}. Best is trial 62 with value: -0.9952445652173914.


{'classifier': 'LGBM', 'num_leaves': 87, 'min_data_in_leaf': 25, 'max_depth': 8, 'lr': 0.1544092064937607, 'max_bin': 106}
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25


In [58]:
print(aucs[i])
print(param[-3:])
print('AUC_BBBP  | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(aucs[i]), np.std(aucs[i])))

[0.7682821081029001, 0.7717506503516716, 0.7617304171885537]
[{'classifier': 'LGBM', 'num_leaves': 69, 'min_data_in_leaf': 14, 'max_depth': 10, 'lr': 0.2034096129886404, 'max_bin': 180}, {'classifier': 'LGBM', 'num_leaves': 39, 'min_data_in_leaf': 26, 'max_depth': 6, 'lr': 0.2538508861901157, 'max_bin': 304}, {'classifier': 'LGBM', 'num_leaves': 87, 'min_data_in_leaf': 25, 'max_depth': 8, 'lr': 0.1544092064937607, 'max_bin': 106}]
AUC_BBBP  | MEAN: 0.7673 | STD: 0.0042


BACE

In [39]:
i = 1
data = datasets[i]
aucs[i] = []

task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)

X = get_vec6(smiles,data=task_name,model_weight=model_weight,hidden_size=256,epoch=epoch,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_classifier, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)

    if study.best_params['classifier']=='LGBM':
        auc = evaluate_classification(LGBMClassifier(num_leaves=study.best_params['num_leaves'], 
                                        min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                        max_depth=study.best_params['max_depth'],
                                        learning_rate=study.best_params['lr'], 
                                        max_bin=study.best_params['max_bin']),
                                        X_train, X_val, X_test, y_train, y_val, y_test)
    
    aucs[i].append(auc)

loading dataset: BACE number of split times: 3


100%|██████████| 1513/1513 [00:00<00:00, 16808.01it/s]

Built smi corpus file!



100%|██████████| 190/190 [00:01<00:00, 126.11it/s]
[I 2023-07-10 11:50:13,763] A new study created in memory with name: no-name-b5afe320-f6ab-4017-9826-a302b43db57f
[I 2023-07-10 11:50:13,910] Trial 0 finished with value: -0.7075337597725657 and parameters: {'classifier': 'LGBM', 'num_leaves': 33, 'min_data_in_leaf': 10, 'max_depth': 3, 'lr': 0.17356320698810135, 'max_bin': 53}. Best is trial 0 with value: -0.7075337597725657.


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=67


...

[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10


[I 2023-07-10 11:52:11,221] Trial 98 finished with value: -0.7151741293532339 and parameters: {'classifier': 'LGBM', 'num_leaves': 30, 'min_data_in_leaf': 10, 'max_depth': 4, 'lr': 0.9609559483139691, 'max_bin': 223}. Best is trial 83 with value: -0.763681592039801.


[LightGBM] [Warning] min_data_in_leaf is set=16, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=16


[I 2023-07-10 11:52:11,998] Trial 99 finished with value: -0.7432480454868515 and parameters: {'classifier': 'LGBM', 'num_leaves': 44, 'min_data_in_leaf': 16, 'max_depth': 10, 'lr': 0.3966397146053252, 'max_bin': 410}. Best is trial 83 with value: -0.763681592039801.


{'classifier': 'LGBM', 'num_leaves': 66, 'min_data_in_leaf': 24, 'max_depth': 6, 'lr': 0.5477019106605963, 'max_bin': 273}
[LightGBM] [Warning] min_data_in_leaf is set=24, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=24


In [40]:
print(aucs[i])
print(param[-3:])
print('AUC_BACE  | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(aucs[i]), np.std(aucs[i])))

[0.7920289855072463, 0.8164855072463768, 0.8014492753623188]
[{'classifier': 'LGBM', 'num_leaves': 90, 'min_data_in_leaf': 34, 'max_depth': 10, 'lr': 0.21414732744127213, 'max_bin': 312}, {'classifier': 'LGBM', 'num_leaves': 96, 'min_data_in_leaf': 48, 'max_depth': 7, 'lr': 0.39597963132279335, 'max_bin': 499}, {'classifier': 'LGBM', 'num_leaves': 66, 'min_data_in_leaf': 24, 'max_depth': 6, 'lr': 0.5477019106605963, 'max_bin': 273}]
AUC_BACE  | MEAN: 0.8033 | STD: 0.0101


multicls

In [65]:
tox21 = dataset.load_Tox21()
sider = dataset.load_SIDER()
clintox = dataset.load_ClinTox()

datasets = [sider, clintox, tox21]

total samples: 7831
total samples: 1427
total samples: 1478


In [66]:
# requires X, y
def objective_classifier_multi(trial):
    model_name = trial.suggest_categorical('classifier', [ 'LGBM']) 
    if model_name ==  'LGBM':
        num_leaves = trial.suggest_int('num_leaves', 5, 100)
        min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 100)
        max_depth = trial.suggest_int('max_depth', -1, 10)
        lr = trial.suggest_loguniform('lr', 1e-5, 1.0)
        max_bin = trial.suggest_int('max_bin', 10, 500)
        clf = LGBMClassifier(num_leaves=num_leaves, min_data_in_leaf=min_data_in_leaf, max_depth=max_depth,
                           learning_rate=lr, max_bin=max_bin, n_jobs=16)
    
    aucs = np.empty(len(KEYS))
    for i,key in enumerate(KEYS):
        _clf = copy.deepcopy(clf)
        _clf.fit(X_train[y_train[key].notna()], y_train[key].dropna().values)
        y_score = _clf.predict_proba(X_val[y_val[key].notna()])
        aucs[i] = roc_auc_score(y_val[key].dropna().values, y_score[:,1])
    return - np.mean(aucs)


from tqdm import tqdm_notebook as tqdm
def evaluate_classification_multi(model, X_train, X_val, X_test):
    aucs = np.empty(len(KEYS))
    for i,key in tqdm(enumerate(KEYS)):
        clf = copy.deepcopy(model)
        _X_train = np.concatenate([X_train[y_train[key].notna()], X_val[y_val[key].notna()]])
        _y_train = np.concatenate([y_train[key].dropna().values, y_val[key].dropna().values])
        clf.fit(_X_train, _y_train)
        y_score = clf.predict_proba(X_test[y_test[key].notna()])
        aucs[i] = roc_auc_score(y_test[key].dropna().values, y_score[:,1])
    return np.mean(aucs)


In [43]:
def vec_ave(data):
    data.columns = range(-1,512,1)
    data1 = data.iloc[:,:257]
    data2 = data.iloc[:,257:]
    data1.columns = range(-1,256,1)
    data1 = data1.rename(columns={-1:'ids'})
    data2.columns= range(0,256,1)
    data2.insert(loc=0, column='ids', value=data[-1])
    data3 = (data1.iloc[:,1:257])/2 + (data2.iloc[:,1:257])/ 2
    DATA = pd.DataFrame(data3)
    return DATA

param = []
aucs = [[],[],[]]

In [44]:
i = 0
data = datasets[i]
aucs[i] = []

task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)
KEYS = Y.keys()


X = get_vec6(smiles,data=task_name,model_weight=model_weight,epoch=epoch,hidden_size=256,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_classifier_multi, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)

    if study.best_params['classifier']=='LGBM':
        auc = evaluate_classification_multi(LGBMClassifier(num_leaves=study.best_params['num_leaves'], 
                                        min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                        max_depth=study.best_params['max_depth'],
                                        learning_rate=study.best_params['lr'], 
                                        max_bin=study.best_params['max_bin']),
                                        X_train, X_val, X_test)


    aucs[i].append(auc)

loading dataset: SIDER number of split times: 3


100%|██████████| 1427/1427 [00:00<00:00, 14104.56it/s]

Built smi corpus file!



100%|██████████| 179/179 [00:01<00:00, 127.44it/s]
[I 2023-07-10 11:52:16,529] A new study created in memory with name: no-name-6dd160b0-bd3b-4897-a8ab-15e9f0724a01


[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
[LightGBM] [Warning] min

...

[LightGBM] [Warning] min_data_in_leaf is set=73, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=73
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] min

[I 2023-07-10 12:35:49,284] Trial 99 finished with value: -0.590763663597219 and parameters: {'classifier': 'LGBM', 'num_leaves': 53, 'min_data_in_leaf': 77, 'max_depth': 3, 'lr': 0.18647487174437558, 'max_bin': 257}. Best is trial 71 with value: -0.6164447501204299.


{'classifier': 'LGBM', 'num_leaves': 59, 'min_data_in_leaf': 76, 'max_depth': 1, 'lr': 0.18974924030760668, 'max_bin': 409}


0it [00:00, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min

In [45]:
print(aucs[i])
print(param[-3:])
print('AUC_SIDER  | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(aucs[i]), np.std(aucs[i])))

[0.6419382119437413, 0.64517925675152, 0.6514465475540752]
[{'classifier': 'LGBM', 'num_leaves': 88, 'min_data_in_leaf': 42, 'max_depth': 0, 'lr': 0.05269185907824778, 'max_bin': 450}, {'classifier': 'LGBM', 'num_leaves': 81, 'min_data_in_leaf': 88, 'max_depth': 8, 'lr': 0.019899814030011124, 'max_bin': 441}, {'classifier': 'LGBM', 'num_leaves': 59, 'min_data_in_leaf': 76, 'max_depth': 1, 'lr': 0.18974924030760668, 'max_bin': 409}]
AUC_SIDER  | MEAN: 0.6462 | STD: 0.0039


In [46]:
i = 2
data = datasets[i]
aucs[i] = []

task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)
KEYS = Y.keys()


X = get_vec6(smiles,data=task_name,model_weight=model_weight,epoch=epoch,hidden_size=256,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_classifier_multi, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)

    if study.best_params['classifier']=='LGBM':
        auc = evaluate_classification_multi(LGBMClassifier(num_leaves=study.best_params['num_leaves'], 
                                        min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                        max_depth=study.best_params['max_depth'],
                                        learning_rate=study.best_params['lr'], 
                                        max_bin=study.best_params['max_bin']),
                                        X_train, X_val, X_test)
    

    aucs[i].append(auc)

loading dataset: Tox21 number of split times: 3


100%|██████████| 7831/7831 [00:00<00:00, 35013.04it/s]


Built smi corpus file!


100%|██████████| 979/979 [00:10<00:00, 92.22it/s] 
[I 2023-07-10 12:38:04,465] A new study created in memory with name: no-name-8450ea54-cd99-4643-8df6-11b790c84422


[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] min

...

[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] min

[I 2023-07-10 13:37:44,457] Trial 99 finished with value: -0.8194644153793987 and parameters: {'classifier': 'LGBM', 'num_leaves': 36, 'min_data_in_leaf': 11, 'max_depth': 10, 'lr': 0.294644054237546, 'max_bin': 195}. Best is trial 52 with value: -0.846172387545444.


{'classifier': 'LGBM', 'num_leaves': 25, 'min_data_in_leaf': 41, 'max_depth': 10, 'lr': 0.06611710590527363, 'max_bin': 259}


0it [00:00, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] min

In [47]:
print(aucs[i])
print(param[-3:])
print('AUC_TOX21 | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(aucs[i]), np.std(aucs[i])))

[0.8329960084349891, 0.8306649032929333, 0.8335680107307041]
[{'classifier': 'LGBM', 'num_leaves': 93, 'min_data_in_leaf': 30, 'max_depth': 9, 'lr': 0.014614549409915335, 'max_bin': 175}, {'classifier': 'LGBM', 'num_leaves': 60, 'min_data_in_leaf': 48, 'max_depth': -1, 'lr': 0.09278385667096922, 'max_bin': 117}, {'classifier': 'LGBM', 'num_leaves': 25, 'min_data_in_leaf': 41, 'max_depth': 10, 'lr': 0.06611710590527363, 'max_bin': 259}]
AUC_TOX21 | MEAN: 0.8324 | STD: 0.0013


In [70]:
i = 1
data = datasets[i]
aucs[i] = []

task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = pd.DataFrame(data.y)
KEYS = Y.keys()


X = get_vec6(smiles,data=task_name,model_weight=model_weight,epoch=epoch,hidden_size=256,n_head=4,n_layer=2)
X_ave = X.copy()

for e,idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx
    X_train = X_ave.loc[train_idx]
    X_val = X_ave.loc[valid_idx]
    X_test = X_ave.loc[test_idx]

    y_train = Y.loc[train_idx]
    y_val = Y.loc[valid_idx]
    y_test = Y.loc[test_idx]

    study = optuna.create_study()
    study.optimize(objective_classifier_multi, n_trials=100, n_jobs=1)
    print(study.best_params)
    param.append(study.best_params)

    if study.best_params['classifier']=='LGBM':
        auc = evaluate_classification_multi(LGBMClassifier(num_leaves=study.best_params['num_leaves'], 
                                        min_data_in_leaf=study.best_params['min_data_in_leaf'], 
                                        max_depth=study.best_params['max_depth'],
                                        learning_rate=study.best_params['lr'], 
                                        max_bin=study.best_params['max_bin']),
                                        X_train, X_val, X_test)


    aucs[i].append(auc)

loading dataset: ClinTox number of split times: 3


100%|██████████| 1478/1478 [00:00<00:00, 20348.47it/s]

Built smi corpus file!



  0%|          | 0/185 [00:00<?, ?it/s]

100%|██████████| 185/185 [00:01<00:00, 128.05it/s]
[I 2023-07-10 16:59:36,556] A new study created in memory with name: no-name-177a81b3-3beb-4614-9414-974d72501072


...

[LightGBM] [Warning] min_data_in_leaf is set=58, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=58
[LightGBM] [Warning] min_data_in_leaf is set=58, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=58


[I 2023-07-10 17:02:37,590] Trial 99 finished with value: -0.9918946301925026 and parameters: {'classifier': 'LGBM', 'num_leaves': 27, 'min_data_in_leaf': 58, 'max_depth': 7, 'lr': 0.6039967856389927, 'max_bin': 264}. Best is trial 73 with value: -0.997467071935157.


{'classifier': 'LGBM', 'num_leaves': 59, 'min_data_in_leaf': 62, 'max_depth': 6, 'lr': 0.7296213723919545, 'max_bin': 335}


0it [00:00, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=62
[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=62


In [71]:
print(aucs[i])
print(param[-3:])
print('AUC_Clintox  | MEAN: {:.4f} | STD: {:.4f}'.format(np.mean(aucs[i]), np.std(aucs[i])))

[0.9994131455399061, 0.9830747478413404, 0.9847271139241942]
[{'classifier': 'LGBM', 'num_leaves': 23, 'min_data_in_leaf': 98, 'max_depth': 9, 'lr': 0.04627540334316179, 'max_bin': 376}, {'classifier': 'LGBM', 'num_leaves': 36, 'min_data_in_leaf': 85, 'max_depth': 8, 'lr': 0.22234759620931197, 'max_bin': 157}, {'classifier': 'LGBM', 'num_leaves': 59, 'min_data_in_leaf': 62, 'max_depth': 6, 'lr': 0.7296213723919545, 'max_bin': 335}]
AUC_Clintox  | MEAN: 0.9891 | STD: 0.0073
